
# Assignment No 2
----
## i160061_SecA_Decision_Tree
## Goal

Your goal in this assigment is to implement a Decision Tree Classifier.

**Note** Please note that you are allowed to use only those libraries which we have discussed in the class, i.e. numpy, scipy, pandas. You can use scikit-learn only for data splitting.

## Submission Instructions
You are required to submit the original notebook file on the Classsroom (with .ipynb extension), with complete set of outputs. Students failing to do so will get zero marks. 

*Please read each step carefully and understand it fully before proceeding with code writing*

## Plagiarism
Any form of plagiarism will not be tolerated and result in 0 marks.



### Decision Tree Classifier

Now in this assignment we will be implementing the Decision Classifier for Continuous attributes.

You will be testing your implementations with only one split criteria, namely:
 - Information Gain


In [1]:
%pylab inline
import scipy.stats
from collections import defaultdict  # default dictionary 
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
class Node:
    def __init__(self,purity,classlabel='',score=0,split=[],fidx=-1):
        self.lchild=None       
        self.rchild=None
        self.classlabel=classlabel        
        self.split=split
        self.score=score #MinEntropy
        self.fidx=fidx #feature index
        self.purity=purity #purity
        
    def set_childs(self,lchild,rchild):
        # YOUR CODE HERE
        self.lchild=lchild
        self.rchild=rchild
        
    def isleaf(self):
        # YOUR CODE HERE
        if self.lchild==None and self.rchild==None:
            return True
        else:
            return False
        
        raise NotImplementedError()
    def isless_than_eq(self, X):
        # YOUR CODE HERE
        raise NotImplementedError()
    def get_str(self):        
        if self.isleaf():
            return 'C(class={},Purity={})'.format(self.classlabel,self.purity)
        else:
            return 'I(Fidx={},Score={},Split={})'.format(self.fidx,self.score,self.split)
    

In [3]:
# A placeholder class 
# TODO: You have to implement the following class, remember from the lectures that you will 
# need to build a model for each different class you are trying to identify...

In [4]:
## Your code goes here...
# You might need to define  classes for composition.. ?
class DecisionTree:
    ''' Implements the Decision Tree For Classification... 
        '''
    def __init__(self, purityp, exthreshold,maxdepth=10):        
        self.purity=purityp
        self.exthreshold=exthreshold #minimun no of examples threshold 
        self.maxdepth=maxdepth
        pass
    def train(self, X, Y):
        ''' Train Decision Tree using the given 
            X [m x d] data matrix and Y labels matrix
            
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional examples.
            Y: [m x 1] a label vector.
            
            Returns:
            -----------
            Nothing
            '''
        
        nexamples,nfeatures=X.shape
        ## now go and train a model for each class...
        # YOUR CODE HERE
        depth=0 
        self.tree = self.build_tree(X,Y,0)
        
        
    def build_tree(self, X, Y, depth):
        # YOUR CODE HERE
        feat_size,total_features = X.shape
        classes = np.unique(Y) #unique classes in dataset
        p_i = np.zeros(classes.size) #to save purity of each class
        for i in range(p_i.shape[0]):
            p_i[i] = Y[Y==classes[i]].shape[0]/Y.shape[0]
        max_purity_class = np.argmax(p_i) #class which purity is max
        max_purity = np.max(p_i) #purity of that class
        newNode = Node(0)
        #base case
        if(max_purity>=self.purity or feat_size <=self.exthreshold or depth >=self.maxdepth):
            newNode.purity = max_purity
            newNode.classlabel = classes[max_purity_class]
            return newNode
        else:
            b_split_point = np.inf
            b_score = np.inf
            b_feat = -1
            for i in range(total_features):
                _split ,_entropy = self.evaluate_numerical_attribute(X[:,i],Y)
                if(_entropy<b_score):
                    b_score = _entropy
                    b_split_point = _split
                    b_feat = i
            #split data on basis of b_split point
            b_split_left_idx = X[:,b_feat]<b_split_point
            b_split_right_idx = X[:,b_feat]>=b_split_point
            
            X_left = X[b_split_left_idx]
            X_Right = X[b_split_right_idx]
            Y_Left = Y[b_split_left_idx]
            Y_Right = Y[b_split_right_idx]
            
            newNode.fidx = b_feat
            newNode.score = b_score
            newNode.split = b_split_point
            newNode.lchild = self.build_tree(X_left,Y_Left,depth+1)
            newNode.rchild = self.build_tree(X_Right,Y_Right,depth+1)
            return newNode
        pass
        
    def test(self, X):
        
        ''' Test the trained classifiers on the given set of examples 
        
                   
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional test examples.
           
            Returns:
            -----------
                pclass: the predicted class for each example, i.e. to which it belongs
        '''
        nexamples, nfeatures=X.shape
        pclasses=self.predict(X)
        return np.array(pclasses)
        pass
    
    def evaluate_numerical_attribute(self,features, Y):
        classes = np.unique(Y)
        nClasses = classes.shape[0]
        sIdx = np.argsort(features)   #sorted index of feature
        nY = np.zeros(Y.shape[0])     #numerical label
        
        for i in range(len(classes)):
            nY[Y==classes[i]] = i
        sFeat = features[sIdx]        #sorted features
        sY = nY[sIdx]                 #sorted labels
        
        mid_points = (sFeat[1:]+sFeat[:-1])/2 #get midpoint of all examples
        
        IG = []
        
        for mid_point in mid_points:
            featLM = sFeat[sFeat<mid_point] #feature less then ith midpoint
            labelLM = sY[0:featLM.shape[0]] # and there labels
            
            featGEM = sFeat[sFeat>=mid_point] #feature greater or equal ith midpoint
            labelGEM = sY[featLM.shape[0]:] # and there labels
            
            pi_l ,pi_r,e_l,e_r = 0,0,0,0
            for i in range(nClasses):
                if(labelLM.shape[0]!=0):
                    pi_l = labelLM[labelLM==i].shape[0]/labelLM.shape[0] #prob of feature at left i.e <midpoint
                if(pi_l!=0):    
                    e_l += pi_l * np.log2(pi_l) #entropy  of feature
                 
                if(labelGEM.shape[0]!=0):
                    pi_r = labelGEM[labelGEM==i].shape[0]/labelGEM.shape[0]
                if(pi_r!=0):
                    e_r += pi_r * np.log2(pi_r)
                
               # print("i:",i,"-->PL:",pi_l,"PR",pi_r)
            
            e_l = -1*e_l
            e_r = -1*e_r
            
            p_l = featLM.shape[0]/features.shape[0]
            p_r = featGEM.shape[0]/features.shape[0]
            
            child_entropy = p_l*e_l + p_r*e_r
            IG.append(child_entropy)
        parent_entropy = 0
        for i in range(nClasses):
            parent_entropy += ((sY[sY==i].shape[0]/sY.shape[0])*np.log2((sY[sY==i].shape[0])/sY.shape[0]))
        parent_entropy = -parent_entropy
        
        minEntropy = np.min(IG)
        iMin = np.argmin(IG)
       # print(parent_entropy-IG)
        return mid_points[iMin], minEntropy+1.0e-20
           
    def predict(self, X):
        predicted=[]
        for i in range(X.shape[0]):
            predicted.append(self._predict(self.tree,X[i,:]))
        return np.array(predicted) 
        
    def _predict(self,node, X):
        if node.isleaf():
            return node.classlabel
        elif X[node.fidx] >= node.split:
            return self._predict(node.rchild, X)
        else:
            return self._predict(node.lchild, X)
    def __str__(self):
        
        return self.__print(self.tree)        
        
     
    def find_depth(self):
        return self._find_depth(self.tree)
    
    
    def _find_depth(self,node):
        if not node:
            return
        if node.isleaf():
            return 1
        else:
            return max(self._find_depth(node.lchild),self._find_depth(node.rchild))+1
    def __print(self,node,depth=0):
        
        ret = ""
        ret += "\n" + ("    "*depth) + node.get_str()
        # Print right branch
        if node.rchild:
            ret += self.__print(node.rchild,depth+1)
        if node.lchild:
            ret += self.__print(node.lchild,depth+1)        
        return ret

In [5]:
import pandas as pd

# In the first case here, you are asked to train on all the data using only two features and classes. 

In [6]:
#load the data set
data=pd.read_csv('./iris.data')
data.columns=['SepalLength','SepalWidth','PetalLength','PetalWidth','Class']
print (data.describe())

       SepalLength  SepalWidth  PetalLength  PetalWidth
count   149.000000  149.000000   149.000000  149.000000
mean      5.848322    3.051007     3.774497    1.205369
std       0.828594    0.433499     1.759651    0.761292
min       4.300000    2.000000     1.000000    0.100000
25%       5.100000    2.800000     1.600000    0.300000
50%       5.800000    3.000000     4.400000    1.300000
75%       6.400000    3.300000     5.100000    1.800000
max       7.900000    4.400000     6.900000    2.500000


In [7]:
# Get your data in matrix
X=np.asarray(data[['SepalLength','SepalWidth','PetalLength','PetalWidth']].dropna())
Y=np.asarray(data['Class'].dropna())
print (" Data Set Dimensions=", X.shape, " True Class labels dimensions", Y.shape ) 

 Data Set Dimensions= (149, 4)  True Class labels dimensions (149,)


In [8]:
# here data is being set so that there are only two classes
Y[Y=='Iris-virginica']='Iris-versicolor'

In [9]:

feat=[0,1] # because we are only using two features in the beginning
dt=DecisionTree(0.95,5,50)
dt.classes=np.unique(Y)
dt.nclasses=len(np.unique(Y))
split,mingain=dt.evaluate_numerical_attribute(X[:,0],Y)
print (split,mingain)

5.45 0.3887071918248902


In [10]:
from nose.tools import assert_almost_equal, assert_equal

assert_equal(split, 5.45)
assert_almost_equal(mingain, 0.38, places=1)

In [11]:
dt.train(X[:,feat],Y)

In [12]:
print (dt)


I(Fidx=0,Score=0.3887071918248902,Split=5.45)
    I(Fidx=1,Score=0.07791297983060938,Split=3.45)
        I(Fidx=0,Score=1e-20,Split=6.5)
            C(class=Iris-versicolor,Purity=1.0)
            C(class=Iris-setosa,Purity=1.0)
        C(class=Iris-versicolor,Purity=1.0)
    I(Fidx=1,Score=0.21622188662602276,Split=2.8)
        C(class=Iris-setosa,Purity=0.9772727272727273)
        I(Fidx=0,Score=1e-20,Split=4.7)
            C(class=Iris-versicolor,Purity=1.0)
            C(class=Iris-setosa,Purity=1.0)


In [13]:
print( dt.find_depth())

4


In [14]:
def XY_Train_Test(X,Y,training_probabilty,random=True):
    no_of_columns = np.shape(X)[1]
    no_of_rows = np.shape(X)[0]
    traning_data_size =int(no_of_rows*training_probabilty)
    temp = np.arange(0,no_of_rows)
    X_Train,Y_Train = np.zeros((traning_data_size,no_of_columns)),np.zeros(traning_data_size,dtype = Y.dtype)
    X_Test,Y_Test = np.zeros((no_of_rows-traning_data_size,no_of_columns)),np.zeros(no_of_rows-traning_data_size,dtype = Y.dtype)
    if(random):
        np.random.shuffle(temp)
    X_Train[0:traning_data_size] = X[temp[0:traning_data_size],]
    X_Test[0:no_of_rows-traning_data_size] = X[temp[traning_data_size:no_of_rows],]
    Y_Train[0:traning_data_size] = Y[temp[0:traning_data_size]]
    Y_Test[0:no_of_rows-traning_data_size] = Y[temp[traning_data_size:no_of_rows]]
    return X_Train,Y_Train,X_Test,Y_Test

In [15]:
# Split your data into training and test-set...
Xtrain,Ytrain,Xtest,Ytest = XY_Train_Test(X,Y,.7)
print (" Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape   )
print (" Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytest.shape   )


 Training Data Set Dimensions= (104, 4) Training True Class labels dimensions (104,)
 Test Data Set Dimensions= (45, 4) Test True Class labels dimensions (45,)


In [18]:
#Lets see how good we are doing, by finding the accuracy on the test set..
pclasses = dt.test(Xtest)
print (np.sum(pclasses==Ytest))
print ("Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0]))

45
Accuracy =  1.0


In [19]:
from nose.tools import assert_greater_equal
feat=[0,1]
dt=DecisionTree(0.95,5,10)
dt.train(Xtrain[:,feat],Ytrain)
pclasses=dt.test(Xtest[:,feat])
acc = np.sum(pclasses==Ytest)/float(Ytest.shape[0])
print(acc)

0.9333333333333333


In [20]:
from nose.tools import assert_greater_equal
feat=[0,1]
dt=DecisionTree(0.95,5)
dt.train(Xtrain[:,feat],Ytrain)
pclasses=dt.test(Xtest[:,feat])
acc = np.sum(pclasses==Ytest)/float(Ytest.shape[0])
print(acc)
assert_greater_equal(acc, 0.90)


0.9333333333333333


# Lets Train on all four features....

In [21]:
#Lets Train on all four features....

# Lets train a Decision Tree Classifier on Petal Length and Width
feat=[0, 1, 2, 3]
dt=DecisionTree(0.95,5)
dt.train(Xtrain[:,feat],Ytrain)
pclasses=dt.test(Xtest[:,feat])
#Lets see how good we are doing, by finding the accuracy on the test set..
print (np.sum(pclasses==Ytest))
print ("Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0]))

45
Accuracy =  1.0


# Lets Train on all four features and for all three classes....

In [22]:
X=np.asarray(data[['SepalLength','SepalWidth','PetalLength','PetalWidth']].dropna())
Y=np.asarray(data['Class'].dropna())
print (" Data Set Dimensions=", X.shape, " True Class labels dimensions", Y.shape)

 Data Set Dimensions= (149, 4)  True Class labels dimensions (149,)


In [23]:
# Split your data into training and test-set... 
# see the documentation of split_data in tools for further information...
Xtrain,Ytrain,Xtest,Ytest = XY_Train_Test(X,Y,.7)
print (" Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape   )
print (" Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytest.shape   )


 Training Data Set Dimensions= (104, 4) Training True Class labels dimensions (104,)
 Test Data Set Dimensions= (45, 4) Test True Class labels dimensions (45,)


In [24]:
feat=[0, 1,2,3]
dt=DecisionTree(0.95,5,50)
good=[0,1]
dt.train(Xtrain[:,good],Ytrain)
pclasses=dt.test(Xtest[:,good])
#Lets see how good we are doing, by finding the accuracy on the test set..
print( np.sum(pclasses==Ytest))
print ("Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0]))

29
Accuracy =  0.6444444444444445


In [25]:
from nose.tools import assert_greater_equal

feat=[0, 1, 2, 3]
dt=DecisionTree(0.95,5)
good=[0,1]
dt.train(Xtrain[:,feat],Ytrain)
pclasses=dt.test(Xtest[:,feat])
#Lets see how good we are doing, by finding the accuracy on the test set..
acc = np.sum(pclasses==Ytest)/float(Ytest.shape[0])
print(acc)
assert_greater_equal(acc, 0.90)


0.9111111111111111


What can you conclude ?
====================
Please write your observation....



## with 2 features and 2 label the accuracy is 91 percent
## with 2 feature and 3 label the accuracy is 77 percent
## with 4 feature and 2 label accuracy is 100 percent
## with 4 feature and 3 label accuracy is 91 percent